In [42]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [43]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [44]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-21 11:00:02,085 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-21 11:00:02,092 INFO: Initializing external client
2025-01-21 11:00:02,094 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-21 11:00:03,434 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [45]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [46]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [47]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [48]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.18s) 


'2025-01-08 15:00:00 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [49]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [50]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  a6d268af35e6fa7ab79f230baa185ea0d6ea9dc7  -  2025-01-20 23:49:13+00:00
Processing commit:  bc1e8fb7d84c26d45d73f9e111c7203d33be3e3d  -  2025-01-20 22:49:02+00:00
Processing commit:  882c182d8fe8bedee7b9af298ae5fac9aa9393fe  -  2025-01-20 21:49:09+00:00
Processing commit:  cc101ffc5b98578177ccca775404f927cc06750f  -  2025-01-20 20:49:08+00:00
Processing commit:  43c551f04c674882940e6a3cf0d279e08f050e34  -  2025-01-20 19:49:03+00:00
Processing commit:  c94aad9c081901d22d9ae0ecfe7d8fda56c81315  -  2025-01-20 18:48:56+00:00
Processing commit:  0026e50a6731b9b35d5d0d84604d14d55e4438e5  -  2025-01-20 17:49:08+00:00
Processing commit:  5264d5a41e58c5842d61ae4bd504bbd1690a9b65  -  2025-01-20 16:49:09+00:00
Processing commit:  0b5f3d45aa9c247b6fad176232c7ad274bef6216  -  2025-01-20 15:49:03+00:00
Processing commit:  63236b7d36d464da4f59a129cc01e5d82eb5697e  -  2025-01-20 14:49:00+00:00
Processing commit:  7d5f24c12af97d075d3e8f99fd1a9425a1d7e0ed  -  2025-01-20 13:49:03+00:00

In [51]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
    df_bike_today['datetime'] = pd.to_datetime(df_bike_today['datetime'], utc=True).dt.floor('H') + datetime.timedelta(hours=1)
df_bike_today


,num_bikes_available,datetime,station
0,40.0,2025-01-21 00:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
1,40.0,2025-01-20 23:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
2,40.0,2025-01-20 22:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
3,40.0,2025-01-20 21:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
4,40.0,2025-01-20 20:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
...,...,...,...
290,40.0,2025-01-08 20:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
291,39.0,2025-01-08 19:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
292,40.0,2025-01-08 18:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
293,40.0,2025-01-08 17:00:00+00:00,HEUSTON_BRIDGE_(NORTH)


## Fetch the weather data for the same time period

In [52]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-21 00:00:00+00:00,6.60,4.344806,0.0,0.0,9.085988,30065.742188,0.0,dublin
1,2025-01-21 01:00:00+00:00,6.50,4.088373,0.0,0.0,10.002560,30065.742188,0.0,dublin
2,2025-01-21 02:00:00+00:00,6.35,3.820613,0.0,0.0,10.805998,30065.742188,0.0,dublin
3,2025-01-21 03:00:00+00:00,6.20,3.544911,0.0,0.0,11.435313,30065.742188,0.0,dublin
4,2025-01-21 04:00:00+00:00,6.10,3.493392,0.0,0.0,10.948973,30065.742188,0.0,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-01-30 19:00:00+00:00,6.55,2.031153,0.0,0.0,18.511143,31849.486328,1.3,dublin
236,2025-01-30 20:00:00+00:00,6.85,2.250813,0.0,0.0,19.645580,31849.486328,1.3,dublin
237,2025-01-30 21:00:00+00:00,7.15,2.459609,0.0,0.0,20.870687,31849.486328,1.3,dublin
238,2025-01-30 22:00:00+00:00,7.45,2.669605,0.0,0.0,22.104116,31849.486328,1.3,dublin


## Insert the bike and weather data into Hopsworks

In [53]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 100.00% |██████████| Rows 295/295 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bike_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [54]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 100.00% |██████████| Rows 240/240 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions
2025-01-21 11:03:42,897 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-21 11:03:46,074 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-21 11:05:18,770 INFO: Waiting for log aggregation to finish.
2025-01-21 11:09:05,682 INFO: Execution finished successfully.
